This Jupyter notebook recreates the distance-decay analyses for the American Gut Project paper by McDonald et al.

Ashkaan K Fahimipour
2016.01.21

In [ ]:
## Let's first install and load some libraries.
install.packages('biom', repos = "http://cran.rstudio.com/")
install.packages('phyloseq', repos = "http://cran.rstudio.com/")
install.packages('qiimer', repos = "http://cran.rstudio.com/")
install.packages('ggplot2', repos = "http://cran.rstudio.com/")
install.packages('vegan', repos = "http://cran.rstudio.com/")
install.packages('fields', repos = "http://cran.rstudio.com/")
install.packages('dplyr', repos = "http://cran.rstudio.com/")
library(biom)
library(phyloseq)
library(qiimer)
library(ggplot2)
library(vegan)
library(fields)
library(dplyr)

## I was given a 1-column csv file with sample IDs to include in my analyses
use <- read.csv('filepath/file.csv', header = TRUE)
use[, 1] <- as.character(use[, 1])

Now we'll read in the BIOM table into R for analysis. R really hates non-JSON formatted BIOM tables (e.g., tables generated with macqiime v1.9), so you may have to use BIOM-convert in to reformat to JSON before importing:

http://biom-format.org/documentation/biom_conversion.html

In [ ]:
## import the table
biom <- read_biom('filepath/file.biom')

## subset data by 'Samples To Use' file
dat <- biom_data(biom)
dat <- as.data.frame(as(biom_data(biom), "matrix"), header = TRUE)
dat <- dat[, which(names(dat) %in% use[, 1])]

## subset again by samples that only occur in >= 10 samples per Josh Ladau's instructions
dat <- dat[-c(which(rowSums(dat != 0) < 10)), ]

## pull out metadata
tax <- as(observation_metadata(biom), "matrix")
meta <- as(sample_metadata(biom), "matrix")
meta <- as.data.frame(samp)

## subset metadata by 'use' rows
meta <- meta[c(which(rownames(meta) %in% use[, 1])), ]

## coordinates matrix for great circle calculation
## I'm assuming column headers weren't renamed, so I'll go with the same for now
coords <- as.matrix(data.frame(lon = as.numeric(as.character(samp$longitude)),
                               lat = as.numeric(as.character(samp$latitude))))

## normalize biom table
dat.rel <- t(t(dat) / rowSums(t(dat))) 
dat.rel <- dat.rel[which(rowSums(dat.rel) > 0), ]

## cube-root transform normalized abundances for well-behaved residuals
dat.rel.trans <- dat.rel^(1/3)

Now that our data are successfully loaded, let's compute our distance matrices.

In [ ]:
## compute Canberra distances
comm.dist <- vegdist(t(dat.rel.trans), method = 'canberra')
comm.dist <- as.matrix(comm.dist)

## compute great circle distance for sites
gc.dist <- rdist.earth(coords, miles = FALSE, R = 6371) ## manually added a more accurate estimate of Earth's radius
colnames(gc.dist) <- colnames(comm.dist)
rownames(gc.dist) <- rownames(comm.dist)

## this is a pretty large dataset, so let's save some memory and
## just plot the upper triangles of both matrices to avoid duplicate points
## in the scatterplot
tri.comm <- comm.dist
diag(tri.comm) <- NA
tri.comm[lower.tri(tri.comm)] <- NA

tri.gc <- gc.dist
diag(tri.gc) <- NA
tri.gc[lower.tri(tri.gc)] <- NA

## bind data for plotting
ggplot.dat <- na.omit(data.frame('comm' = c(tri.comm), 'gc' = c(tri.gc)))

## distance-decay plot for all data
ggplot(ggplot.dat, aes(x = gc, y = comm)) + 
  geom_point(colour = 'dodgerblue4', alpha = 0.7, size = 0.35) +
  stat_smooth(colour = 'black', method = "lm", 
              formula = y ~ poly(x, 1)) + ## fits a linear model
  guides(colour = FALSE, fill = FALSE) +
  xlab('Spatial Distance (km)') +
  ylab('Community Distance') +
  theme_bw() + 
  theme(panel.grid.minor = element_blank(), panel.grid.major = element_blank(), 
        panel.border = element_blank(), panel.grid.minor = element_blank(), 
        panel.background = element_blank()) +
  theme(axis.line = element_line(color = 'black'))

## and we perform a permutational test to assess significance
mant.mod <- mantel(comm.dist, gc.dist, method = 'spear', permutations = 999)
mant.mod

The next section performs individual analyses at the state-level, and performs Benjamini-Hochberg correction for multiple comparisons and spits them out into a summary table, so that you can evaluate which states (if any) display significant distance-decay relationships and plot those particular states, if you want.

In the first version of the data I recieved, I had to manually clean up multiple discrepancies in state namings (e.g., state labels for Colorado were inputted as both 'CO' and 'Colorado'. Not sure if this has been corrected in the new data. If not, let me know and we can figure out a quick solution.

In [ ]:
## define some things beforehand
uni.state <- unique(samp$state)
state.frames <- list() ## save the distance outputs for later
mantel.list <- list() ## save statistics for later

for(s in 1:length(uni.state)){
  curr.state <- uni.state[s]
  meta.state <- as.data.frame(meta)
  meta.state <- subset(meta.state, state == curr.state)
  
  ## subset data by state
  dat.state <- dat[, which(names(dat) %in% rownames(meta.state))]
  
  ## pull out lat and lon coordinates
  coords.state <- data.frame(lon = samp.state[, 'longitude'], lat = samp.state[, 'latitude'])
  rownames(coords.state) <- NULL
  coords.state$lon <- as.numeric(as.character(coords.state$lon)) ## make sure R doesn't do any of its weird factor stuff
  coords.state$lat <- as.numeric(as.character(coords.state$lat))
  coords.state <- as.matrix(coords.state)
  
  ## normalize new community matrix
  dat.rel.state <- t(t(dat.state) / rowSums(t(dat.state)))
  dat.rel.state <- dat.rel.state[which(rowSums(dat.rel.state) > 0), ]
  dat.rel.state.trans <- dat.rel.state^(1/3)  
  canb.state <- vegdist(t(dat.rel.state), method = 'canberra') ## compute state-level distance matrix
  canb.state <- as.matrix(canb.state)
  
  ## compute great circle distance for sites
  gc.state <- rdist.earth(coords.state, miles = FALSE, R = 6371)
  
  ## mantel test
  mant.state <- mantel(canb.state, gc.state, method = 'spear', permutations = 999)
  mantel.list[[s]] <- c(as.character(curr.state), 
                             mant.state$statistic, ## correlation
                             mant.state$signif) ## p-value
  )
  
  ## take upper triangle of both matrices
  tri.canb.state <- canb.state
  diag(tri.canb) <- NA
  tri.canb.state[lower.tri(tri.canb.state)] <- NA
  tri.gc.state <- gc.state
  diag(tri.gc.state) <- NA
  tri.gc.state[lower.tri(tri.gc.state)] <- NA
  
  ## save data for plotting and fit model
  state.frames[[s]] <- na.omit(data.frame('comm' = c(tri.canb.state), 'gc' = c(tri.gc.state), 'state' = rep(curr.state)))
  
  ## progress bar to drop us a line
  if(s == 1){bar.vec <- c(na.omit(seq(1:length(uni.state))[1:length(uni.state) * round(length(uni.state) / 10)]))
  cat('|')}
  if(f %in% bar.vec == TRUE){cat('=====|')}
}

## bind these lists together
state.dat <- do.call('rbind', state.frames)
mantel.frame <- as.data.frame(do.call('rbind', mantel.list))
names(mantel.frame) <- c('state', 'coef', 'p')

## adjust p-values with FDR
mantel.frame$p.adjust <- p.adjust(mantel.frame$p, method = 'fdr')

You should now have a table called 'mantel.frame' summarizing state-level distance-decay relationships. To plot specific states, run the cell below after specifying which states you want to visualize.

In [ ]:
## subset data by states of interest. for instance, CO and CA.
state.dat.sub <- subset(state.dat, state == 'CO' | state == 'CA')

## quick ggplot
ggplot(state.dat.sub, aes(x = gc, y = comm, show_guide = FALSE)) + 
  geom_point(alpha = 0.8, size = .6, colour = 'dodgerblue4') +
  stat_smooth(width = 5, colour = 'black', method = "lm", formula = y ~ x, linetype = 2, level = 0, fill = 'dodgerblue4') +
  xlab('Great Circle Distance (km)') +
  ylab('Community Distance') +
  guides(colour = FALSE, fill = FALSE) +
  theme_bw() + 
  theme(panel.grid.minor = element_blank(), panel.grid.major = element_blank(), 
        panel.border = element_blank(), panel.grid.minor = element_blank(), 
        panel.background = element_blank()) +
  theme(axis.line = element_line(color = 'black')) +
  facet_wrap(~ state)